In [ ]:
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
import os

# --- KONFIGURACJA ---
INPUT_CSV = "data/ecoli_only_100000.csv"  # Plik z sORFami
OUTPUT_CSV = "positives_with_upstream.csv"
UPSTREAM_LEN = 30

In [ ]:
def load_genome(fasta_path):
    """Wczytuje genom do słownika {accession: sekwencja}"""
    print(f"Wczytywanie genomu z {fasta_path}...")
    genome_dict = {}
    for record in SeqIO.parse(fasta_path, "fasta"):
        # Pobieramy czysty identyfikator (do pierwszej spacji)
        acc = record.id
        genome_dict[acc] = str(record.seq).upper()
    print(f"Wczytano {len(genome_dict)} kontigów/sekwencji.")
    return genome_dict


def get_upstream_offline(genome_dict, accession, sorf_seq, upstream_len=30):
    """Szuka sORF w lokalnym genomie i wycina upstream"""
    if accession not in genome_dict:
        return ""

    full_seq = genome_dict[accession]
    target_sorf = sorf_seq.upper()

    # 1. Szukamy na nici dodatniej (+)
    pos = full_seq.find(target_sorf)
    if pos != -1:
        start_pos = pos
        if start_pos >= upstream_len:
            return full_seq[start_pos - upstream_len : start_pos]
        else:
            # Jeśli sORF jest na samym początku genomu
            return full_seq[0:start_pos].zfill(upstream_len).replace("0", "N")

    # 2. Szukamy na nici ujemnej (-)
    # Jeśli sORF jest na nici ujemnej, to w genomie (nić +)
    # znajduje się jego Reverse Complement (RC)
    rc_sorf = str(Seq(target_sorf).reverse_complement())
    pos_rc = full_seq.find(rc_sorf)

    if pos_rc != -1:
        # Na nici ujemnej "upstream" znajduje się ZA sekwencją RC na nici dodatniej
        # 5' [Upstream] [sORF] 3' (nić -)
        # 3' [RC Upstream] [RC sORF] 5' (nić +)
        end_pos_rc = pos_rc + len(rc_sorf)
        rc_upstream = full_seq[end_pos_rc : end_pos_rc + upstream_len]

        # Musimy odwrócić RC_upstream, aby dostać sekwencję 5'->3' dla nici ujemnej
        return str(Seq(rc_upstream).reverse_complement())

    return ""  # Nie znaleziono dopasowania

In [ ]:
for genome_file in os.listdir("data/ecoli_genomes/"):

    if not os.path.exists("data/ecoli_genomes/" + genome_file):
        print(
            f"BŁĄD: Nie znaleziono pliku genomu: {"data/ecoli_genomes/" + genome_file}"
        )

    # Wczytanie danych
    genome_dict = load_genome("data/ecoli_genomes/" + genome_file)
    df = pd.read_csv(INPUT_CSV)

    # print(f"Przetwarzanie {len(df)} rekordów...")

    # Ekstrakcja upstream
    results = []
    for idx, row in df.iterrows():
        upstream = get_upstream_offline(
            genome_dict, row["accession"], row["sorf"], UPSTREAM_LEN
        )
        results.append(upstream)

        # if (idx + 1) % 1000 == 0:
        #     print(f"Przetworzono {idx + 1} rekordów...")

    df["upstream_30bp"] = results

    # Zapis
df.to_csv(OUTPUT_CSV, index=False)
print(f"Sukces! Wyniki zapisano w: {OUTPUT_CSV}")